In [1]:
import sys
sys.path.append('..')

In [2]:
import numpy as np
from lab2.AES import encrypt, decrypt
from lab2.ByteGF import BitGF
from lab2.utils import *
from copy import deepcopy

# AES - 128

<img src = "Screenshot from 2020-02-27 19-44-20.png">

In [3]:
inp = np.array([
        BitGF.from_hex('32'),
        BitGF.from_hex('43'),
        BitGF.from_hex('f6'),
        BitGF.from_hex('a8'),
        BitGF.from_hex('88'),
        BitGF.from_hex('5a'),
        BitGF.from_hex('30'),
        BitGF.from_hex('8d'),
        BitGF.from_hex('31'),
        BitGF.from_hex('31'),
        BitGF.from_hex('98'),
        BitGF.from_hex('a2'),
        BitGF.from_hex('e0'),
        BitGF.from_hex('37'),
        BitGF.from_hex('07'),
        BitGF.from_hex('34'),
    ]).reshape((4, 4)).transpose()

key = np.array([
    BitGF.from_hex('2b'),
    BitGF.from_hex('7e'),
    BitGF.from_hex('15'),
    BitGF.from_hex('16'),
    BitGF.from_hex('28'),
    BitGF.from_hex('ae'),
    BitGF.from_hex('d2'),
    BitGF.from_hex('a6'),
    BitGF.from_hex('ab'),
    BitGF.from_hex('f7'),
    BitGF.from_hex('15'),
    BitGF.from_hex('88'),
    BitGF.from_hex('09'),
    BitGF.from_hex('cf'),
    BitGF.from_hex('4f'),
    BitGF.from_hex('3c'),
]).reshape((4, 4)).transpose()

print(inp, '\n\n', key)

[[32 88 31 e0]
 [43 5a 31 37]
 [f6 30 98 07]
 [a8 8d a2 34]] 

 [[2b 28 ab 09]
 [7e ae f7 cf]
 [15 d2 15 4f]
 [16 a6 88 3c]]


<img src = 'Screenshot from 2020-02-27 19-50-04.png'>

In [4]:
enc = encrypt(inp, key)
enc

array([[39, 02, dc, 19],
       [25, dc, 11, 6a],
       [84, 09, 85, 0b],
       [1d, fb, 97, 32]], dtype=object)

In [5]:
decrypted = decrypt(enc, key)
decrypted

array([[32, 88, 31, e0],
       [43, 5a, 31, 37],
       [f6, 30, 98, 07],
       [a8, 8d, a2, 34]], dtype=object)

In [6]:
decrypted == inp

array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])

# Як зміниться криптотекст якщо в початковому тексті замінити один біт?

In [7]:
changed_inp = deepcopy(inp)

In [8]:
changed_inp[2][2].bits[4] = not changed_inp[2][2].bits[4]

In [9]:
print(inp, '\n\n', changed_inp)

[[32 88 31 e0]
 [43 5a 31 37]
 [f6 30 98 07]
 [a8 8d a2 34]] 

 [[32 88 31 e0]
 [43 5a 31 37]
 [f6 30 88 07]
 [a8 8d a2 34]]


In [10]:
enc = encrypt(inp, key)
enc

array([[39, 02, dc, 19],
       [25, dc, 11, 6a],
       [84, 09, 85, 0b],
       [1d, fb, 97, 32]], dtype=object)

In [11]:
changed_bit_enc = encrypt(changed_inp, key)
changed_bit_enc

array([[40, 1b, 8d, 00],
       [60, 77, 67, 1d],
       [6c, b6, 5b, fc],
       [eb, 01, 3b, 6f]], dtype=object)

In [12]:
enc == changed_bit_enc

array([[False, False, False, False],
       [False, False, False, False],
       [False, False, False, False],
       [False, False, False, False]])

Як видно з останньго виводу криптотексти не  співпадають в жодному байті.

# Потокові режими

## CBC

In [13]:
from lab2.stream_mods import cbc_encrypt, cbc_decrypt

In [14]:
text = "Test message for testing CBC stream mod."

test_key = next(text2byte_generator("TEST KEY FOR ME."))

encoded = np.array([*text2byte_generator(text), ])

In [15]:
enc = np.array([*cbc_encrypt(iter(encoded), test_key), ])
print(byte2text(enc))

"ëeMN¾ÜËé	îòH©'Ê9­¶Y×~HëJq¬)ý¢Uþ	h»Çb>2+'ûxsjÉüJ


In [16]:
decr = np.array([*cbc_decrypt(iter(enc), test_key), ])
print(byte2text(decr))

Test message for testing CBC stream mod.


In [17]:
print(decr == encoded)

[[[ True  True  True  True]
  [ True  True  True  True]
  [ True  True  True  True]
  [ True  True  True  True]]

 [[ True  True  True  True]
  [ True  True  True  True]
  [ True  True  True  True]
  [ True  True  True  True]]

 [[ True  True  True  True]
  [ True  True  True  True]
  [ True  True  True  True]
  [ True  True  True  True]]]


## CTR

In [18]:
from lab2.stream_mods import ctr_encrypt, ctr_decrypt

In [19]:
text = "Test message for testing CTR stream mod."

test_key = next(text2byte_generator("TEST KEY FOR CTR"))

encoded = np.array([*text2byte_generator(text), ])

In [20]:
enc = np.array([*ctr_encrypt(iter(encoded), test_key), ])
print(byte2text(enc))

ôÕËäÿÑI»WÚx·ÄÝãGû·zWÏc·ÅÑÕ°^ý^¢ô.Þw¼Å


In [21]:
decr = np.array([*ctr_decrypt(iter(enc), test_key), ])
print(byte2text(decr))

Test message for testing CTR stream mod.


In [22]:
print(decr == encoded)

[[[ True  True  True  True]
  [ True  True  True  True]
  [ True  True  True  True]
  [ True  True  True  True]]

 [[ True  True  True  True]
  [ True  True  True  True]
  [ True  True  True  True]
  [ True  True  True  True]]

 [[ True  True  True  True]
  [ True  True  True  True]
  [ True  True  True  True]
  [ True  True  True  True]]]
